In [2]:
import numpy as np
import pandas as pd
import tqdm
from scipy import sparse
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix

# Prepare 

Создается новый датасет с заполненной new_id
- если у юзера есть user_id -> user_id = new_id
- если у юзера нет user_id -> user_id = cookie_id
- у юзеров, где пропущены user_id заполняются пропуски

In [ ]:
df = pd.read_parquet('processed_train_full.parquet', engine='pyarrow')

In [ ]:
df.drop_duplicates(inplace=True)

In [ ]:
users_cookies = df[df['user_id'].notna()].groupby(['cookie_id'], as_index=False)['user_id'].agg(['unique'])
users_cookies_list = users_cookies.index

def set_id_unknown(cookie):
    if cookie in users_cookies_list:
        return users_cookies.loc[cookie][0][0]
    else:
        return cookie
    
df['new_id'] = df.apply(lambda x: set_id_unknown(x['cookie_id']), axis = 1)
print('Нет пропусков id = ', df[df['new_id'].isna()]['new_id'].sum() == 0)

In [ ]:
df.to_csv('with_new_ids_final.csv', index=False)

In [4]:
df = pd.read_csv('with_new_ids_final.csv')

In [ ]:
# default vacancies to give
top_vacancies = df.pivot_table(index='vacancy_id_', columns='event_type', values='event_timestamp', aggfunc='count', fill_value=0)
top_vacancies['ctr'] = top_vacancies.apply(lambda x: (x['preview_click_vacancy'] + x['click_response'] + x['click_contacts'] + x['preview_click_response']+ x['click_favorite'] + x['preview_click_favorite'] + x['preview_click_contacts'] + x['click_phone'] + x['preview_click_phone'])/x['show_vacancy'] if x['show_vacancy'] != 0 else 0, axis=1)
top_vacancies = list(top_vacancies.sort_values(['ctr'], ascending=False).index[:10])

# Векторизация юзеров

- юзеры представляются в виде векторов: юзер х колонки вакансий,

0 - если юзер не сделал по вакансии ключевых действий, иначе 1.

In [ ]:
# Создается массив векторов
def create_massive(original_df, filename):
    uniq_users_id = original_df['new_id'].unique()
    uniq_vacancy = original_df['vacancy_id_'].unique()
    size = len(uniq_vacancy)
    
    interactions = original_df.query('event_type != "preview_click_vacancy" and event_type != "show_vacancy"') \
    .groupby(['new_id', 'vacancy_id_', 'event_type'], as_index=False) \
    .agg({'event_timestamp': 'count'})

    massive = csr_matrix((len(uniq_users_id), size), dtype = np.int8)
    for k1, i in enumerate(tqdm.tqdm(uniq_users_id)):
        user_vacancies = set(interactions[interactions['new_id'] == i]['vacancy_id_'])
        for k2, w in enumerate(uniq_vacancy):
            if w in user_vacancies: massive[k1,k2] = 1 # юзер - вакансия

    sparse.save_npz('{}.npz'.format(filename), massive)
    return massive

In [ ]:
# Get vectorized users
sparse_vectors = create_massive(df, 'final_clean_massive')

# Косинус схожести

- находим похожих по ключевым действиям юзеров

In [ ]:
# Рассчитывается косинус схожести для sparse массива, в итоге получаем похожих юзеров в виде координат
similarity = cosine_similarity(sparse_vectors, dense_output=False)
sparse.save_npz('similarity_results.npz', changed_similarity)

In [ ]:
A = pd.DataFrame.sparse.from_spmatrix(similarity)
display(A)

# Функции для получения потенциальных вакансий для рекомендации

In [ ]:
# куки и соответствующие id юзеров
users_cookies = df.groupby(['cookie_id'], as_index=False)['new_id'].agg(['unique'])

# тут уже все действия юзеров
interactions = df.groupby(['new_id', 'vacancy_id_', 'event_type'], as_index=False) \
.agg({'event_timestamp': 'count'}) \
.sort_values(['event_timestamp'], ascending=False)

In [ ]:
# Получить new_id/user_id по куки
def get_new_id_by_cookie(cookie):
    return users_cookies.loc[cookie][0][0]

# Лист всех уникальных юзеров по-порядку, в котором составлялась матрица
user_unique_columns = df['new_id'].unique()
# Получить индекс юзера в списке
def get_user_index(user_id):
    return np.where(user_unique_columns == user_id)[0][0]

In [ ]:

# Получение вакансий для рекомендации
def get_similar_users_ordered(user_index):
    similar_users_index_list = list()
    for i, val in enumerate(A[user_index]):
        if (val > 0.3) and (i != user_index):
            similar_users_index_list.append((i, val))
    similar_users_index_list = sorted(similar_users_index_list, key = lambda x: x[1], reverse=True)
    return similar_users_index_list

# Ранжирование товаров юзера
def get_vacancies_ordered_by_user(new_id):
    user_interaction = interactions[interactions['new_id'] == new_id]
    unique_user_events = user_interaction['event_type'].unique()

    vacancy_table = user_interaction.pivot_table(index='vacancy_id_', columns='event_type', values='event_timestamp', fill_value=0).reset_index()
    vacancy_table['ctr'] = vacancy_table.apply(lambda x: (sum([x[i] for i in unique_user_events])), axis=1)
    vac_dict = list([[v['vacancy_id_'] , v['ctr']] for i, v in vacancy_table.iterrows()])
    return sorted(vac_dict, key = lambda x: x[1], reverse=True)

In [ ]:
# Получение отранжированных вакансий по всем юзерам
def get_ordered_vacancies(similar_users_list):
    recommend_vac = list()
    for i in similar_users_list[:10]: # пока беру первых 5 похожих юзеров
        user = user_unique_columns[i[0]]
        vacancies = get_vacancies_ordered_by_user(user)
        for v in vacancies:
            recommend_vac.append(v[0])
    recommend_vac = list(dict.fromkeys(recommend_vac))
    return recommend_vac


# Получить использованные вакансии
def get_used_vacancies(new_id):
    return interactions[interactions['new_id'] == new_id]['vacancy_id_'].unique()

In [ ]:
# Получение рекомендации + в рекомендации учитваем топ вакансий
def get_user_recommendation(user_id):
    recommend_result = []
    index = get_user_index(user_id)
    similar_index_list = get_similar_users_ordered(index)
    
    all_recommended_vac = get_ordered_vacancies(similar_index_list)
    users_used_vacs = get_used_vacancies(user_id)
    
    if len(all_recommended_vac) > 0:
        for i in all_recommended_vac:
            if i not in users_used_vacs:
                recommend_result.append(i)
    else:
        return top_vacancies
    recommend_result = top_vacancies + recommend_result
    return recommend_result

# Чтение тестового датасета

In [ ]:
test_df = pd.read_parquet('test_public_mfti.parquet', engine='pyarrow')
test_df

In [ ]:
def get_new_id_by_cookie(cookie):
    return users_cookies.loc[cookie][0][0]

In [ ]:
test_df['cookie_id'] = test_df['cookie_id'].apply(lambda x: get_new_id_by_cookie(x))
test_df = test_df.rename(columns={'cookie_id': 'new_id'})

In [ ]:
test_df

# Добавляем потенциальные вакансии для юзеров из тестового датасета

In [ ]:
# Для тестового датасета добавляем колонку pot_vacancies, где будут указаны потенциальные рекомендации на основе косинуса схожести
def make_pred_vacs_col(idx):
    pot_vacs = get_user_recommendation(idx)
    return pot_vacs

test_df['pot_vacancies'] = test_df.apply(lambda x: make_pred_vacs_col(x['new_id']), axis=1)

In [ ]:
# Преобразовываем датафейм для использования модели, которая сранжирует потенциальные вакансии
# test_df['pot_vacancies'] - потенциальные
users = []
vacancies = []

for i in test_df.itertuples():
    users = users + ([i[1]] * len(i[3]))
    vacancies = vacancies + i[3]
    
new_test = pd.DataFrame({'new_id': users, 'vacancy_id_': vacancies})

In [ ]:
new_test.to_csv('new_test_final.csv', index=False)